## INFERENCE NOTEBOOK

In [1]:
# load modules

import os
import tensorflow as tf 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import numpy as np
from skimage import transform

In [2]:
SEED = 1234
tf.random.set_seed(SEED) 

## Predicting from a directory 

In [3]:
# our root dataset dir path
data_dir = "../dataset/"

In [4]:
# load model
MODEL_PATH = '../trained_model_imagenet'

model = tf.keras.models.load_model(MODEL_PATH)

In [6]:
# the total classes
num_classes=7

# the labels indexes
classes = [
    'boots', 
    'flip_flops',
    'loafers',
    'sandals',
    'sneakers',
    'soccer_shoes',
    'no_shoe'
           ]

In [7]:
# Test data real-time augmentation configuration that does normalization
valid_data_gen = ImageDataGenerator(rescale=1./255)

In [8]:
# Instantiating generators of augmented image batches (and their labels)
test_dir = os.path.join(data_dir, 'val')
test_gen = valid_data_gen.flow_from_directory(test_dir,
                                           target_size=(256, 256),
                                           classes=classes,
                                           class_mode='categorical',
                                           shuffle=False,
                                           seed=SEED)

Found 350 images belonging to 7 classes.


In [9]:
# reset the data generator
test_gen.reset()

# predict on the data
pred = model.predict(test_gen,verbose=1)

11/11 [==============================] - 213s 18s/step


In [10]:
# get filenames of the data images
filenames = test_gen.filenames

In [11]:
# get predicted indexes
predicted_class_indices = np.argmax(pred,axis=1)

# get predicted labels
predictions = [k for k in predicted_class_indices]

outputs = [classes[ind] for ind in predictions]

In [12]:
# create a dataframe with the filenames and output labels

out_list = [[filename,output] for filename,output in zip(filenames, outputs)]

df = pd.DataFrame(out_list, columns=['Filename', 'Predicted Label'])

df.head()

,Filename,Predicted Label
0,boots\280-bootsshoe.png,boots
1,boots\281-bootsshoe.png,boots
2,boots\282-bootsshoe.png,boots
3,boots\283-bootsshoe.png,boots
4,boots\284-bootsshoe.png,boots


In [13]:
# svae the dataframe to a csv

df.to_csv('../predictions/predictions.csv',index=False)

## Predicting for a list of  Images

In [14]:
def load(filename):
    '''
        Input: ImagePath
        Output : Process image array
    '''
    np_image = Image.open(filename) # open image
    np_image = np.array(np_image).astype('float32')/255 # normalize image
    np_image = transform.resize(np_image, (256, 256, 3)) # resize image
    np_image = np.expand_dims(np_image, axis=0)
    return np_image

In [15]:
def classify_image(image_paths,model,classes):
    '''
        Input: List of image local paths or URLs
        Output : List of predicted labels
    '''
    
    response = []
    for image_path in image_paths:
        image = load(image_path) # process image
        preds = model.predict(image) # predict label index
        output = classes[np.argmax(preds,axis=1)[0]] # get the label
        response.append(output)
    return response

In [16]:
image_paths = ['..\\dataset\\val\\boots\\280-bootsshoe.png','..\\dataset\\val\\boots\\281-bootsshoe.png']
classify_image(image_paths,model,classes)

['boots', 'boots']